<a href="https://colab.research.google.com/github/JRCon1/Option-Greek-Interactive-Plotting/blob/main/GreekPlots.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
%%capture
!pip install py_vollib

# Since I am using Google Colab, I need to manually import any atypical packages, and I use the %%capture to hide the output of the package download to avoid clutter

from py_vollib.black_scholes.greeks.analytical import delta, gamma, vega, theta, rho
import pandas as pd
import yfinance as yf
import numpy as np
from datetime import datetime, timedelta
import plotly.graph_objects as go
from scipy.stats import norm
import matplotlib.pyplot as plt

r = 0.04 # Predefined a risk-free rate of 0.04. This can be changed here and will remain constant throughout the code
upper = lower = float(input('Enter the upper & lower bound for strikes as a decimal (Ex: 0.20 for 20%): ')) # Enter in the limit of bounds for the strikes as %. IE, if you only want options from 90 to 110 of a $100 stock, pick 0.10, or a 10% move
ticker_symbol = input('Enter the ticker symbol: ').upper() # Pick a ticker/company
opt_type = input('Enter the option type (c for call, p for put): ').lower() # Puts or Calls?
ticker = yf.Ticker(ticker_symbol)
current_price = ticker.history(period="1y")['Close'].iloc[-1]
lower_bound = round(current_price * (1 - lower), 0)
upper_bound = round(current_price * (1 + upper), 0)

# Define function for our baseline filters on the initial Data Query
def filter_opts_within_range(ticker, expiry_dates, lower_bound, upper_bound):
    filtered_options = pd.DataFrame()
    relevant_columns = ['strike', 'lastPrice', 'bid', 'ask', 'volume', 'openInterest', 'impliedVolatility', 'expiry']
    for expiry in expiry_dates:
      if opt_type == 'c':
        calls = ticker.option_chain(expiry).calls
        calls_filtered = calls[(calls['strike'] >= lower_bound) & (calls['strike'] <= upper_bound)].copy()
        calls_filtered['expiry'] = expiry
        filtered_options = pd.concat([filtered_options, calls_filtered[relevant_columns]], ignore_index=True)
      elif opt_type == 'p':
        puts = ticker.option_chain(expiry).puts
        puts_filtered = puts[(puts['strike'] >= lower_bound) & (puts['strike'] <= upper_bound)].copy()
        puts_filtered['expiry'] = expiry
        filtered_options = pd.concat([filtered_options, puts_filtered[relevant_columns]], ignore_index=True)
    return filtered_options

filtered_options = filter_opts_within_range(ticker, ticker.options, lower_bound, upper_bound) # Apply the function
max_expiry_date = datetime.now() + timedelta(weeks=16) # Define max expiry
filtered_options['expiry'] = pd.to_datetime(filtered_options['expiry']) # Apply datetime function to expiry
filtered_options = filtered_options[filtered_options['expiry'] <= max_expiry_date] # Use max_expiry_date variable from previous to filter once again
filtered_options['Days To Expiry'] = (filtered_options['expiry'] - datetime.now()).dt.days + 2 # Define Days to Expiry and adjust for package error
filtered_options['Cost'] = (filtered_options['strike'] - filtered_options['lastPrice']) * 100 # Define cost (for Option Sellers primarily)
filtered_options = filtered_options[filtered_options['Days To Expiry'] > 0] # Get rid of 0 DTE Options (optional)
filtered_options['Moneyness'] = filtered_options['strike'].apply(lambda strike: 'ITM' if (opt_type == 'p' and strike > current_price) or (opt_type == 'c' and strike < current_price) else 'OTM') # Define Moneyness
filtered_options['Mid-Point Price'] = round((filtered_options['bid'] + filtered_options['ask']) / 2, 4) # Define Mid-Point price of Bid and Ask

# Calculate the Intrinsic Value of the Option
def calculate_intrinsic_value(row):
    return 0 if row['Moneyness'] == 'OTM' else row['strike'] - current_price
filtered_options['Intrinsic Value'] = filtered_options.apply(calculate_intrinsic_value, axis=1) * 100 # Apply function and adjust for Net Value
filtered_options['Premium Per Day (Extrinsic)'] = round(((filtered_options['lastPrice'] * 100) - filtered_options['Intrinsic Value']) / filtered_options['Days To Expiry'], 4) # Calculate Premium Per day (Theta Decay over lifetime of option)
filtered_options['Return on Capital as %'] = round((((filtered_options['lastPrice'] * 100) - filtered_options['Intrinsic Value']) / filtered_options['Cost']) * 100, 4) # Return on Capital (For Option Sellers)

# Function to calculate all greeks at once (rounded to the 4th)
def greeks(row, option_type=opt_type, r=r):
    S = current_price
    K = row['strike']
    t = row['Days To Expiry'] / 365
    sigma = row['impliedVolatility']
    delta_value = abs(round(delta(option_type, S, K, t, r, sigma), 4))
    gamma_value = round(gamma(option_type, S, K, t, r, sigma), 4)
    vega_value = round(vega(option_type, S, K, t, r, sigma), 4)
    theta_value = abs(round(theta(option_type, S, K, t, r, sigma), 4))
    rho_value = abs(round(rho(option_type, S, K, t, r, sigma), 4))
    return pd.Series({'Delta': delta_value, 'Gamma': gamma_value, 'Vega': vega_value, 'Theta': theta_value, 'Rho': rho_value})

filtered_options[['Delta', 'Gamma', 'Vega', 'Theta', 'Rho']] = filtered_options.apply(greeks, axis=1) #Apply Functiona for Greeks

Enter the upper & lower bound for strikes as a decimal (Ex: 0.20 for 20%): 0.1
Enter the ticker symbol: UPRO
Enter the option type (c for call, p for put): p


In [7]:
# Define the Function to calculate the greeks
def calculate_greeks(row, option_type=opt_type, r=0.04):
    S=current_price
    K=row['strike']
    t=row['Days To Expiry']/365
    sigma=row['impliedVolatility']
    if t<=0 or sigma<=0:
        return {'Delta':0.0,'Gamma':0.0,'Vega':0.0,'Theta':0.0,'Rho':0.0}
    delta_value=abs(round(delta(option_type,S,K,t,r,sigma),4))
    gamma_value=round(gamma(option_type,S,K,t,r,sigma),4)
    vega_value=round(vega(option_type,S,K,t,r,sigma),4)
    theta_value=abs(round(theta(option_type,S,K,t,r,sigma),4))
    rho_value=abs(round(rho(option_type,S,K,t,r,sigma),4))
    return {'Delta':delta_value,'Gamma':gamma_value,'Vega':vega_value,'Theta':theta_value,'Rho':rho_value}

# Define function to calculate day-over-day changes in Greeks
def greekChange(data, greek_col, group_col):
    data=data.sort_values(by=[group_col,'Days To Expiry'],ascending=[True,False])
    data[f'{greek_col}_Sequential_Change']=data.groupby(group_col)[greek_col].pct_change()*100
    return data.sort_values(by=[group_col,'Days To Expiry'],ascending=[True,True]).reset_index(drop=True)

simulated_data_global = None

# Define function to simulate the greeks (highlights days as actual options that are NOT estimates)
def simulate_greeks_with_highlighted_days(filtered_options):
    unique_strikes=sorted(filtered_options['strike'].unique())
    greeks=['Delta','Gamma','Vega','Theta','Rho']
    colors=['blue','green','purple','red','orange']
    fixed_days=np.arange(1,121)
    unique_days=filtered_options['Days To Expiry'].unique()
    simulated_data=[]
    for greek in greeks:
        filtered_options=greekChange(filtered_options,greek_col=greek,group_col='strike')
    for strike in unique_strikes:
        strike_data=filtered_options[filtered_options['strike']==strike].iloc[0]
        for dte in fixed_days:
            simulated_row=strike_data.copy()
            simulated_row['Days To Expiry']=dte
            greek_values=calculate_greeks(simulated_row)
            simulated_data.append({'strike':strike,'Days To Expiry':dte,**greek_values})
    simulated_df=pd.DataFrame(simulated_data)
    for greek in greeks:
        simulated_df=greekChange(simulated_df,greek_col=greek,group_col='strike')
    fig=go.Figure()
    for idx,strike_price in enumerate(unique_strikes):
        for greek,color in zip(greeks,colors):
            strike_data=simulated_df[simulated_df['strike']==strike_price]
            fig.add_trace(go.Scatter(x=strike_data['Days To Expiry'],y=strike_data[greek],mode='lines',line=dict(color=color),name=f"{greek} (Strike {strike_price})",hovertext=[f"Strike: {strike_price}<br>{greek}: {val:.2f}<br>Change: {change:.2f}%<br>Days to Expiry: {dte}" for val,change,dte in zip(strike_data[greek],strike_data[f'{greek}_Sequential_Change'],strike_data['Days To Expiry'])],visible=True))
            fig.add_trace(go.Scatter(x=strike_data[strike_data['Days To Expiry'].isin(unique_days)]['Days To Expiry'],y=strike_data[strike_data['Days To Expiry'].isin(unique_days)][greek],mode='markers',marker=dict(size=8,symbol='circle',color=color),name=f"{greek} Markers (Strike {strike_price})",hovertext=[f"Strike: {strike_price}<br>{greek}: {val:.2f}<br>Change: {change:.2f}%<br>Days to Expiry: {dte}" for val,change,dte in zip(strike_data[strike_data['Days To Expiry'].isin(unique_days)][greek],strike_data[strike_data['Days To Expiry'].isin(unique_days)][f'{greek}_Sequential_Change'],strike_data[strike_data['Days To Expiry'].isin(unique_days)]['Days To Expiry'])],visible=True))
    strike_dropdown=[{"method":"update","label":"All Strikes","args":[{"visible":[True]*len(fig.data)},{"title":"Simulated Greeks with Highlighted Days for All Strikes"}]}]+[{"method":"update","label":f"Strike {strike_price}","args":[{"visible":[(i//(len(greeks)*2)==idx) for i in range(len(unique_strikes)*len(greeks)*2)]},{"title":f"Simulated Greeks with Highlighted Days for Strike {strike_price}"}]} for idx,strike_price in enumerate(unique_strikes)]
    greek_dropdown=[{"method":"update","label":"All Greeks","args":[{"visible":[True]*len(fig.data)},{"title":"All Greeks with Highlighted Days Across All Strikes"}]}]+[{"method":"update","label":greek,"args":[{"visible":[(i%(len(greeks)*2)==greek_idx*2 or i%(len(greeks)*2)==greek_idx*2+1) for i in range(len(unique_strikes)*len(greeks)*2)]},{"title":f"{greek} with Highlighted Days Across All Strikes"}]} for greek_idx,greek in enumerate(greeks)]
    fig.update_layout(title="Simulated Greeks with Highlighted Days",xaxis_title="Days to Expiry",yaxis_title="Greek Value",legend_title="Greeks and Strikes",updatemenus=[{"buttons":strike_dropdown,"direction":"down","showactive":True,"x":0.15,"y":1.15,"xanchor":"left","yanchor":"top"},{"buttons":greek_dropdown,"direction":"down","showactive":True,"x":0.4,"y":1.15,"xanchor":"left","yanchor":"top"}],template="plotly_white")
    fig.update_xaxes(autorange="reversed",title_text="Days to Expiry (Inverted)")
    fig.show()
    global simulated_data_global
    simulated_data_global = simulated_df

# Return plot with our previous generated options chain
simulate_greeks_with_highlighted_days(filtered_options)